In [1]:
#!pip install "langchain-community==0.2.12"

In [2]:
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain.chat_models import ChatOpenAI
import os
from google.colab import userdata

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Set OpenAI key
openai_api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

In [6]:
# Load data
path = '/content/drive/MyDrive/Final Project BBS/data/'
file_name = 'steam_reviews_summarizer.csv'
df = pd.read_csv(f"{path}{file_name}", encoding = "latin1")
df['date'] = pd.to_datetime(df['date'])
df.shape

(9182, 13)

# Select reviews to summarize

In [7]:
# Set input parameters
game_input = "Apex Legends"
sentiment = "Negative"
n_reviews = 5

In [8]:
# Filter reviews
selected_reviews = (
    df[(df['word_count'] >= 5) &
     (df['game'] == game_input) &
      (df['sentiment_category'] == sentiment)]
    .sort_values("date", ascending=False)
    .head(n_reviews))

In [9]:
# Get review text
reviews_text = "\n\n".join(
        f"- {row.clean_review}" for _, row in selected_reviews.iterrows()
    )

# AI summarizer agent

In [10]:
# Create the model
llm = ChatOpenAI(
    model="gpt-4.1",
    temperature=0.2,
    openai_api_key=openai_api_key
)

/tmp/ipython-input-3131004674.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(


In [11]:
# Prompt for summarization

template = """
You are an AI assistant specializing in summarizing game reviews.
Summarize the following user-selected reviews focusing on sentiment, main issues, recurring themes, and overall user experience.

Reviews:
{reviews}

Provide a concise but informative summary.
"""
prompt = PromptTemplate.from_template(template)

In [12]:
# Call model
response = llm([HumanMessage(content=prompt.format(reviews=reviews_text))])

/tmp/ipython-input-1028976799.py:2: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = llm([HumanMessage(content=prompt.format(reviews=reviews_text))])


In [14]:
# Print model output
print(response.content)

**Summary of User Reviews:**

**Sentiment:**  
Overall sentiment is strongly negative. Users express disappointment and frustration with the game's current state.

**Main Issues:**
- **Ownership Concerns:** Some users are unhappy with the game's new ownership, associating it with negative connotations.
- **Declining Quality:** Multiple reviews mention that the game was better in the past and has since become "garbage."
- **Ranked System Changes:** There is significant dissatisfaction with recent changes to the ranked mode, particularly:
  - Loss of control over starting locations, increasing randomness and unfairness.
  - Kill participation no longer contributing meaningfully to rank progression.
  - Gameplay now favors hiding and survival over combat skills, making it less engaging for players who prefer action.
  - The ranked system feels tailored to professional players, alienating regular users.
- **Technical Issues:** Frequent crashes are reported, severely impacting playability.
